In [2]:
#Importing Dependencies
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
#ETL FOR ALTERNATIVE DATA SOURCE, IGNORE THIS ONE!
# books_df= pd.read_csv("books.csv", on_bad_lines='skip')
# books_df.head()

In [3]:
#ETL FOR BX-Books, change position A1 to 'ISBN' in the CSV in Excel!
#I had to change the encoding because the CSV would not read in with UTF-08.
BX_1= pd.read_csv("/Users/juliencounts/Desktop/Data_Analysis/Classes/Final_Project_4/Resources/BX-Books.csv", on_bad_lines='skip', encoding='latin-1' )
BX_1.head()

/var/folders/xn/zxr8x63d7dl9t22nbdnj_b2c0000gn/T/ipykernel_72118/2050309160.py:3: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  BX_1= pd.read_csv("/Users/juliencounts/Desktop/Data_Analysis/Classes/Final_Project_4/Resources/BX-Books.csv", on_bad_lines='skip', encoding='latin-1' )


,ISBN,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,"0195153448;""Classical Mythology"";""Mark P. O. M...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"0002005018;""Clara Callan"";""Richard Bruce Wrigh...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"0060973129;""Decision in Normandy"";""Carlo D'Est...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"0374157065;""Flu: The Story of the Great Influe...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"0393045218;""The Mummies of Urumchi"";""E. J. W. ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#Splitting the ISBN column based on the semi-colon
BX_1_split= BX_1['ISBN'].str.split(';')
BX_1_split.head()

0    [0195153448, "Classical Mythology", "Mark P. O...
1    [0002005018, "Clara Callan", "Richard Bruce Wr...
2    [0060973129, "Decision in Normandy", "Carlo D'...
3    [0374157065, "Flu: The Story of the Great Infl...
4    [0393045218, "The Mummies of Urumchi", "E. J. ...
Name: ISBN, dtype: object

In [6]:
#Turning this split into a dataframe
BX_1_df= pd.DataFrame(BX_1_split.tolist()).add_prefix('Book_')
BX_1_df.head()

,Book_0,Book_1,Book_2,Book_3,Book_4,Book_5,Book_6,Book_7,Book_8,Book_9,Book_10,Book_11,Book_12,Book_13
0,0195153448,"""Classical Mythology""","""Mark P. O. Morford""","""2002""","""Oxford University Press""","""http://images.amazon.com/images/P/0195153448....","""http://images.amazon.com/images/P/0195153448....","""http://images.amazon.com/images/P/0195153448....",None,None,None,None,None,None
1,0002005018,"""Clara Callan""","""Richard Bruce Wright""","""2001""","""HarperFlamingo Canada""","""http://images.amazon.com/images/P/0002005018....","""http://images.amazon.com/images/P/0002005018....","""http://images.amazon.com/images/P/0002005018....",None,None,None,None,None,None
2,0060973129,"""Decision in Normandy""","""Carlo D'Este""","""1991""","""HarperPerennial""","""http://images.amazon.com/images/P/0060973129....","""http://images.amazon.com/images/P/0060973129....","""http://images.amazon.com/images/P/0060973129....",None,None,None,None,None,None
3,0374157065,"""Flu: The Story of the Great Influenza Pandemi...","""Gina Bari Kolata""","""1999""","""Farrar Straus Giroux""","""http://images.amazon.com/images/P/0374157065....","""http://images.amazon.com/images/P/0374157065....","""http://images.amazon.com/images/P/0374157065....",None,None,None,None,None,None
4,0393045218,"""The Mummies of Urumchi""","""E. J. W. Barber""","""1999""","""W. W. Norton &amp","Company""","""http://images.amazon.com/images/P/0393045218....","""http://images.amazon.com/images/P/0393045218....","""http://images.amazon.com/images/P/0393045218....",None,None,None,None,None


In [7]:
#Removing the unnecessary columns
BX_1_drop= BX_1_df.drop(columns=['Book_5','Book_6', 'Book_7','Book_8','Book_9', 
                                      'Book_10', 'Book_11', 'Book_12', 'Book_13'])
BX_1_drop.head()

,Book_0,Book_1,Book_2,Book_3,Book_4
0,0195153448,"""Classical Mythology""","""Mark P. O. Morford""","""2002""","""Oxford University Press"""
1,0002005018,"""Clara Callan""","""Richard Bruce Wright""","""2001""","""HarperFlamingo Canada"""
2,0060973129,"""Decision in Normandy""","""Carlo D'Este""","""1991""","""HarperPerennial"""
3,0374157065,"""Flu: The Story of the Great Influenza Pandemi...","""Gina Bari Kolata""","""1999""","""Farrar Straus Giroux"""
4,0393045218,"""The Mummies of Urumchi""","""E. J. W. Barber""","""1999""","""W. W. Norton &amp"


In [8]:
#Renaming the columns. This is the final step for BX-Books.csv
BX_1 = BX_1_drop.rename(columns={'Book_0': 'ISBN', 'Book_1': 'Title', 'Book_2': 'Author', 
                                        'Book_3': 'Year', 'Book_4': 'Publisher'})
BX_1.head()

,ISBN,Title,Author,Year,Publisher
0,0195153448,"""Classical Mythology""","""Mark P. O. Morford""","""2002""","""Oxford University Press"""
1,0002005018,"""Clara Callan""","""Richard Bruce Wright""","""2001""","""HarperFlamingo Canada"""
2,0060973129,"""Decision in Normandy""","""Carlo D'Este""","""1991""","""HarperPerennial"""
3,0374157065,"""Flu: The Story of the Great Influenza Pandemi...","""Gina Bari Kolata""","""1999""","""Farrar Straus Giroux"""
4,0393045218,"""The Mummies of Urumchi""","""E. J. W. Barber""","""1999""","""W. W. Norton &amp"


In [9]:
#Reading in the BX-Books-Ratings.csv. Change position A1 to Book-Rating in Excel to make this easy!
BX_ratings= pd.read_csv("/Users/juliencounts/Desktop/Data_Analysis/Classes/Final_Project_4/Resources/BX-Book-Ratings.csv",
                        on_bad_lines='skip', encoding='latin-1')
BX_ratings.head()

/var/folders/xn/zxr8x63d7dl9t22nbdnj_b2c0000gn/T/ipykernel_72118/2970673956.py:2: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  BX_ratings= pd.read_csv("/Users/juliencounts/Desktop/Data_Analysis/Classes/Final_Project_4/Resources/BX-Book-Ratings.csv",


,Book-Rating,Unnamed: 1,Unnamed: 2
0,"276725;""034545104X"";""0""",NaN,NaN
1,"276726;""0155061224"";""5""",NaN,NaN
2,"276727;""0446520802"";""0""",NaN,NaN
3,"276729;""052165615X"";""3""",NaN,NaN
4,"276729;""0521795028"";""6""",NaN,NaN


In [10]:
#Splitting the Book-Rating column based on the semi-colon
BX_ratings_split= BX_ratings['Book-Rating'].str.split(';')
BX_ratings_split.head()

0    [276725, "034545104X", "0"]
1    [276726, "0155061224", "5"]
2    [276727, "0446520802", "0"]
3    [276729, "052165615X", "3"]
4    [276729, "0521795028", "6"]
Name: Book-Rating, dtype: object

In [11]:
#Pushing the split to a dataframe
BX_ratings_df= pd.DataFrame(BX_ratings_split.tolist()).add_prefix('Col_')
BX_ratings_df.head()

,Col_0,Col_1,Col_2
0,276725,"""034545104X""","""0"""
1,276726,"""0155061224""","""5"""
2,276727,"""0446520802""","""0"""
3,276729,"""052165615X""","""3"""
4,276729,"""0521795028""","""6"""


In [12]:
#Renaming the columns
BX_ratings= BX_ratings_df.rename(columns={'Col_0': 'User_ID', 'Col_1': 'ISBN_1', 'Col_2': 'Rating'})
BX_ratings.head()

,User_ID,ISBN_1,Rating
0,276725,"""034545104X""","""0"""
1,276726,"""0155061224""","""5"""
2,276727,"""0446520802""","""0"""
3,276729,"""052165615X""","""3"""
4,276729,"""0521795028""","""6"""


In [13]:
#Dropping the apostrophes around ISBN_1 and Rating so it will join with the first CSV 
BX_ratings['ISBN_1']= BX_ratings['ISBN_1'].str.replace(r'"', '')
BX_ratings['Rating']=BX_ratings['Rating'].str.replace(r'"', '')
BX_ratings.head()

,User_ID,ISBN_1,Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [14]:
#Dropping the null values in the ratings dataframe
BX_ratings.replace('NULL', np.nan, inplace=True)
BX_ratings.dropna(inplace=True)

#Converting the ratings column
BX_ratings['Rating']= BX_ratings['Rating'].astype('int')

BX_ratings.head()

,User_ID,ISBN_1,Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [15]:
#Reading in the BX-Users
BX_users= pd.read_csv('/Users/juliencounts/Desktop/Data_Analysis/Classes/Final_Project_4/Resources/BX-Users.csv', 
                      on_bad_lines='skip', encoding='latin-1')
BX_users= BX_users.drop(columns=['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'])
BX_users.head()

/var/folders/xn/zxr8x63d7dl9t22nbdnj_b2c0000gn/T/ipykernel_72118/688142864.py:2: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  BX_users= pd.read_csv('/Users/juliencounts/Desktop/Data_Analysis/Classes/Final_Project_4/Resources/BX-Users.csv',


,City,State,Country
0,"1;""nyc",new york,"usa"";NULL"
1,"2;""stockton",california,"usa"";""18"""
2,"3;""moscow",yukon territory,"russia"";NULL"
3,"4;""porto",v.n.gaia,"portugal"";""17"""
4,"5;""farnborough",hants,"united kingdom"";NULL"


In [16]:
#Getting rid of the apostraphes around cities and countries
BX_users['City']= BX_users['City'].str.replace(r'"', '')
BX_users['Country']=BX_users['Country'].str.replace(r'"', '')
BX_users.head()

,City,State,Country
0,1;nyc,new york,usa;NULL
1,2;stockton,california,usa;18
2,3;moscow,yukon territory,russia;NULL
3,4;porto,v.n.gaia,portugal;17
4,5;farnborough,hants,united kingdom;NULL


In [17]:
#Cleaning up the whole Users dataframe
BX_users.isnull().sum()
BX_users_clean= BX_users.dropna()
BX_users_clean.isnull().sum()

City       0
State      0
Country    0
dtype: int64

In [18]:
#Making the state column a dataframe
BX_users_state=pd.DataFrame(BX_users_clean['State'].tolist())
BX_users_state.columns=['State']

#Fixing the index for concatenation 
BX_users_state.reset_index(drop=True)
BX_users_state.head()

,State
0,new york
1,california
2,yukon territory
3,v.n.gaia
4,hants


In [19]:
#Splitting City
BX_users_split= BX_users_clean['City'].str.split(';')
BX_users_split.head()

0            [1, nyc]
1       [2, stockton]
2         [3, moscow]
3          [4, porto]
4    [5, farnborough]
Name: City, dtype: object

In [20]:
#Pushing the city split to a dataframe
BX_users_df= pd.DataFrame(BX_users_split.tolist()).add_prefix('Col_')
BX_users_city=BX_users_df.drop(columns=['Col_0','Col_2','Col_3','Col_4','Col_5','Col_6','Col_7','Col_8','Col_9']).rename(columns={'Col_1':'City'})
BX_users_city.head()

,City
0,nyc
1,stockton
2,moscow
3,porto
4,farnborough


In [21]:
#Splitting Country 
BX_country_split= BX_users_clean['Country'].str.split(';')
BX_country_split.head()

0               [ usa, NULL]
1                 [ usa, 18]
2            [ russia, NULL]
3            [ portugal, 17]
4    [ united kingdom, NULL]
Name: Country, dtype: object

In [22]:
#Pushing the country split to a dataframe and renaming the columns country and age
BX_country_split_df= pd.DataFrame(BX_country_split.tolist()).add_prefix('Col_')
BX_country_split_df= BX_country_split_df.drop(columns=['Col_2','Col_3','Col_4']).rename(columns={'Col_0':'Country', 'Col_1': 'Age'})
BX_country_split_df.head()


,Country,Age
0,usa,NULL
1,usa,18
2,russia,NULL
3,portugal,17
4,united kingdom,NULL


In [23]:
#Concatenated for a complete users dataframe!
BX_users= pd.concat([BX_users_city, BX_users_state, BX_country_split_df], axis=1)

#Removing those pesky null values again
BX_users.replace('NULL', np.nan, inplace=True)
BX_users.dropna(inplace=True)

BX_users.head(20)

,City,State,Country,Age
1,stockton,california,usa,18
3,porto,v.n.gaia,portugal,17
5,santa monica,california,usa,61
9,albacete,wisconsin,spain,26
10,melbourne,victoria,australia,14
12,barcelona,barcelona,spain,26
17,rio de janeiro,rio de janeiro,brazil,25
18,weston,,,14
19,langhorne,pennsylvania,usa,19
20,ferrol / spain,alabama,spain,46


In [24]:
#Getting an overview of the cleaned users dataframe
BX_users.describe()

,City,State,Country,Age
count,168165,168165,168165,168165
unique,24952,4337,310,2714
top,barcelona,california,usa,24
freq,2519,9498,75514,5599


In [25]:
#Joining the ratings csv with the books csv based on matching their ISBNs.
BX_complete= pd.merge(BX_ratings, BX_1, left_on="ISBN_1", right_on="ISBN", how="inner")
BX_complete.head(20)

,User_ID,ISBN_1,Rating,ISBN,Title,Author,Year,Publisher
0,276725,034545104X,0,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""
1,2313,034545104X,5,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""
2,6543,034545104X,0,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""
3,8680,034545104X,5,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""
4,10314,034545104X,9,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""
5,23768,034545104X,0,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""
6,28266,034545104X,0,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""
7,28523,034545104X,0,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""
8,39002,034545104X,0,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""
9,50403,034545104X,9,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""


In [26]:
#Getting rid of the ISBN_1 column since it's no longer needed
BX_complete= BX_complete.drop(columns=['ISBN_1'])

In [27]:
#Final clean up! 
BX_complete_clean= BX_complete.dropna()
BX_complete_clean.isnull().sum()
BX_complete_clean.head(10)

,User_ID,Rating,ISBN,Title,Author,Year,Publisher
0,276725,0,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""
1,2313,5,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""
2,6543,0,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""
3,8680,5,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""
4,10314,9,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""
5,23768,0,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""
6,28266,0,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""
7,28523,0,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""
8,39002,0,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""
9,50403,9,034545104X,"""Flesh Tones: A Novel""","""M. J. Rose""","""2002""","""Ballantine Books"""


In [28]:
#Exporting clean data as CSV
from pathlib import Path 
filepath = Path('/Users/juliencounts/Desktop/Data_Analysis/Classes/Final_Project_4/Resources/BX_final.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
BX_complete_clean.to_csv(filepath)  

In [29]:
#Getting an overview of the final dataset's ratings
BX_complete_clean.describe()

,Rating
count,834178.000000
mean,2.881100
std,3.865598
min,0.000000
25%,0.000000
50%,0.000000
75%,7.000000
max,10.000000


In [30]:
BX=BX_complete_clean
BX_top_ten=BX.groupby('Title')['Rating'].agg(['mean', 'count'])

In [31]:
wonder= BX_top_ten.sort_values(by='count',ascending=False)
wonder.head(10)

,mean,count
Title,,
"""Wild Animus""",1.033156,2262
"""The Lovely Bones: A Novel""",4.457045,1164
"""The Da Vinci Code""",4.669492,826
"""A Painted House""",3.199739,766
"""The Nanny Diaries: A Novel""",3.459816,759
"""Bridget Jones's Diary""",3.518919,740
"""The Secret Life of Bees""",4.558239,704
"""Divine Secrets of the Ya-Ya Sisterhood: A Novel""",3.455904,669
"""The Red Tent (Bestselling Backlist)""",4.320840,667


In [32]:
jack= wonder['count'].sort_values('counts'>100, ascending=False)
jack.head()

TypeError: '>' not supported between instances of 'str' and 'int'